# Part 1: Measuring the Z boson mass
 This notebook contains basic anlaysis as described in the lab manual.

# Step 0. Importing useful packages

 Most python scripts start with 'import' statements like this that make useful packages available. Examples are 'uproot' which reads in our data files and allows them to be read, 'numpy' is a powerful and popular package for fast manipultation of arrays and 'scipy.stats' is useful for statitical analysis,we'll use it to generate and fit functions to our data.

In [1]:
import lorentz
import uproot3 as uproot
import uproot_methods.classes.TLorentzVector as LVepm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import cauchy
from scipy.stats import norm

import pandas as pd

# Step 1. Access the data file and convert into a 'pandas dataframe'.

 - Here the file is converted into the very poweful 'pandas dataframe' format which is used a lot in modern
     statistical analysis. You can think of the dataframe as a super-charged spreadsheet. Each row represents 
     a proton-proton collision event, and every column represents a piece of information ('observable') we have
     recorded from that event.

 - We are going to work with 7 observables: the components of the muon's four-vector, the charge of the muon and two     extra variables related to how isolated the muon is from other particles. As we will have two muons produced in       each event, that makes 14 observables per event.
 
 
The first thing we want to do is read our data file. Remebmber it contains ~500k proton collision events from the ATLAS Open Data pre-selected so that each event contains two muon candidates with transverse mometum greater than 20000 MeV.

In [3]:
eventsData = uproot.open("data_Skim_mumu.root")["mini"] #this command tells uproot where to find the file
df = eventsData.pandas.df(["muon_E", "muon_pt", "muon_phi", "muon_eta", "muon_charge", "muon_etcone20", "muon_ptcone30"])
print("File has been successfully opened!")

FileNotFoundError: [Errno 2] No such file or directory: 'data_Skim_mumu.root'

# NOTE - NONE OF THE REST OF THIS NOTEBOOK WILL RUN UNTIL YOU DOWNLOAD THE INPUT DATA AND SIMULATION FILES FROM HERE. https://drive.google.com/drive/folders/1iqUqc277CDBXsfUks0Z3i9EzhH0C5_Fp?usp=sharing

 - You will need to be logged in to google with your UCT account to gain access
 - PLACE ALL FILES IN THE SAME DIRECTORY AS THIS NOTEBOOK.

We said dataframes are like spreadsheets and we can see that structure directly by simply typing the name of the variable representing the dataframe ("df") into the prompt.
We'll see the familiar row/column structure with our twelve observables and lots of events!

In [ ]:
df

You will see each event has values of the variables for "muon[0]" and "muon[1]". These refer to the muon
with the larger and smaller pt values respectively.

The next thing we want to do is apply some more critertia to our data so that it is dominated as much as possible by the Z->mumu process while still retaining as many events as possible.

# Step 2. Apply event selections

In [ ]:
#cut0: we require the charges of the muons to be opposite (by requiring the sum to be 0)
cuts0 = df[ ((df["muon_charge[0]"]) + (df["muon_charge[1]"])  == 0)   ]

#cut1: we require the transervse momentum of each muon to be above 20000 MeV'
cuts1 = cuts0[(cuts0["muon_pt[0]"] > 20000) & (cuts0["muon_pt[1]"] > 20000 )]

#cut2: we require the absolute pseudorapidity of each muon to be below 2.4'
cuts2 = cuts1[( np.abs(cuts1["muon_eta[0]"]) < 2.4) & ( np.abs(cuts1["muon_eta[1]"]) < 2.4)]

# cuts 2&3: we require the muons to be isolated from other particles in the event these criteria will be
# explored later in step 7, for now leave them commented out.

cuts3 = cuts2[( (cuts2["muon_etcone20[0]"])/(cuts2["muon_pt[0]"]) < 0.1) & ( (cuts2["muon_etcone20[1]"])/(cuts2["muon_pt[1]"]) < 0.1) ]
cuts4 = cuts3[( (cuts3["muon_ptcone30[0]"])/(cuts3["muon_pt[0]"]) < 0.1) & ( (cuts3["muon_ptcone30[1]"])/(cuts3["muon_pt[1]"]) < 0.1) ]

#we copy out finally selected dataframe to a new variable 'finalData' for convenience
finalData =  cuts4 # you will have to change this line when you want to inlcude the muon 
#isolation criteria, i.e., "finalData =  cuts4" 

#let's check how many events we have selected after all our criteria have been applied
print("Number of selected events = " + str(len(finalData.index)))

# and have a look at our finally selected dataframe
finalData

 # Step 3. 'Reconstruct' the Z boson by adding the four-vectors of each muon.

In [ ]:
#first we make arrays of four-vectors for the muons. we make separate arrays for muon[0] (leading) and muon[1] (sub-leading)
lvArray0 = LVepm.TLorentzVectorArray.from_ptetaphi(finalData["muon_pt[0]"], finalData["muon_eta[0]"], finalData["muon_phi[0]"],finalData["muon_E[0]"])
lvArray1 = LVepm.TLorentzVectorArray.from_ptetaphi(finalData["muon_pt[1]"], finalData["muon_eta[1]"], finalData["muon_phi[1]"],finalData["muon_E[1]"])

# the TLorentzVectorArray class conveniently allows us to simply add the array
# to get an array of four-vectors representing the dimuon system in each event
lvArray = lvArray0 + lvArray1
lvArray

# Step 4.  Make histogram plots of the kinematic information of the muon pairs.

In [ ]:
# let's make some simple histograms of the kineamtic information associated with the muons.
# as an example I look at muon[0] only, but you can look at muon[1] and compare the distributions.

# plot the mu[0] pt histogram.
plt.figure()
plt.xlabel("mu pt")
plt.ylabel("events per bin")
plt.hist(lvArray0.pt, bins=35, range=[0,120000], alpha=0.6, color='g')

# plot the mu[0] pseudorapdity histogram.
plt.figure()
plt.xlabel("mu eta")
plt.ylabel("events per bin")
plt.hist(lvArray0.eta, bins=25, range=[-3.0,3.0], alpha=0.6, color='g')

# plot the mu[0] phi histogram.
plt.figure()
plt.xlabel("mu phi")
plt.ylabel("events per bin")
plt.hist(lvArray0.phi, bins=15, range=[-3.14,3.14], alpha=0.6, color='g')

# Step 5. Make histogram plots of the kinematic information of the muons.
- now that we have reconstructed four-vectors of the dimuon system and know how to make histogram plots,let's make a histogram plot of the dimuon mass distribution and see if the Cauchy distribution can be seen and if the peak is somehwhere around 90 GeV

- we will compare this histogram to a cauchy pdf. However, pdfs are always normalised to an area of 1 while our data represents hundreds of thousands of events, so the data histogram will surely have an integral much larger than 1. Therefore we must first calculate this integral use it to scale our pdf to allow a fair visual comparison between the data histogram and the predicted Cauchy distribution.

In [ ]:
#we set up a few parmaters for our histogram, min and max of the x-axis (mass)
minMass = 71000
maxMass = 110000

#the number of bins
nBins = 250 

# creating the histogram as two arrays (bin edges & counts in the bins) numpy
countsData, edges = np.histogram(lvArray.mass, bins=nBins, range=(minMass, maxMass))

# get the width of each bin
bin_width = edges[1] - edges[0]
# sum over number in each bin and mult. by bin width, which can be factored out.
# This gives us the integral
integral = bin_width * sum(countsData[0:nBins])

# we can make an array of the centre of each bin directly from the edges array this will be useful in plotting our pdf
centres = (edges[1:] + edges[:-1]) / 2

#fit a Cauchy distributions to the dimuon mass data
#mu, std = cauchy.fit(lvArray.mass)
#print("mu = " + str(mu))
#print("std = " + str(std))

So we are almost ready to compare our data histogram with the predicted Cauchy distribution but we know the Cauchy pdf has two free parameters: the mean (mu) and standard deviation (sigma). what should we choose for these values for a first comparison? Why not the world-average values published by the Particle Data group below...

In [ ]:
mZPDG  = 91187.6
sigmaZPDG = 2495.2

#let's make the comparison plot
plt.figure()
p = (cauchy.pdf(centres, mZPDG, sigmaZPDG) * integral)
plt.plot(centres, p, 'k',label="Cauchy pdf (PDG values)", linewidth=2)
plt.hist(lvArray.mass, bins=nBins, range=[minMass, maxMass],label="ATLAS Data", alpha=0.6, color='g')
plt.legend()
plt.xlabel("mumu mass [MeV]")
plt.ylabel("# events")
plt.show()

The plot above shows a clear peak and a shape that looks very much like a cauchy distrubtion. So we can be confident that our data conatains the decays of a heavy particle to pairs of muons. However the data does not perfectly agree with the predicted shape of a cauchy distributions with the PDG values for the mass and width of the Z boson.



# 6. Fitting the Cauchy model to the data
In the next part of this notebook we will try to assess if we can find values of the Z mass which result in a better 'agreement' between the data and cauchy model. If we trust the cauchy model completely, then this proceedure is a measurement of the z mass. We'll define the agreement between our data and model with the chi-squared function. the function take in arrays of events counts ('obs') and predictions ('preds') corresponding to the bins of the dimuon mass distribution. the function returns the squared difference between the event count and prediction divded by the variance on the prediction, summed over all bins, otherwise known as the chi-squared function.

In [ ]:
#chi-squared function
def calcChiSq(obs, preds):
    chiSq = 0.0
    ndf = len(obs)
    for bin in range(0, len(obs)):
        diff = preds[bin] - obs[bin]
        #print("diff = " + str(diff))
        #var = ( np.abs(obs[bin])) # neymans's chi2 (approximate the of the data's variance as poisson)
        var = ( np.abs(preds[bin])) # pearson's chi2 (approximate the of the prediction's variance as poisson)
        if (var != 0):
            chiSq += (diff**2)/(var)
            #print("obs, pred, diff, var  chi contrib = "+  str( obs[bin]) +" " +  str(preds[bin]) +" "+  str(diff) +" "+  str(var) +" " + str((diff**2)/(var)))
    return chiSq, ndf

To perform the fit we will:
- We'll now loop over a range of $m_{Z}$ values, 
- Generate a cauchy distribution for each value evaluate the $\chi^{2}$ between that distribution and the data histogram.
- find the $m_{Z}$ value that gives the smallest the value of the $\chi^{2}$
- plot the $\Delta \chi^{2}$ vs the $m_{Z}$, and use the \emph{critical values} of this curve to evulated the uncertainties on $m_{Z}$
- compare the data histogram to the fitted} Cauchy distribution

In [ ]:
#some arrays to hold the z mass and chi-squared values
mZAr = np.array([])
chi2Ar = np.array([])

#deciding the range of z mass values and how fine the steps in this range will be
minMz = 90730
maxMz = 90765
step = 1

#vairables to use in the loop to find value of z mass that gives smallest chi2
bestFitMz1D = 0.0
minChi2 = 10000000

for mZ in range(minMz, maxMz, step): #starting loop
    countsPDF = (cauchy.pdf(centres, mZ, sigmaZPDG) * integral) #building pdf for these values
    #countsPDF = (norm.pdf(centres, mZ, sigmaZPDG) * integral) #building pdf for these values
    chi2, ndf = calcChiSq(countsData, countsPDF) # call chi2 function to get chi2 for this particular pdf and data
    mZAr = np.append(mZAr, mZ) #adding values to arrays
    chi2Ar = np.append(chi2Ar, chi2)
    if(chi2 < minChi2): #keeping track of what is the smallest chi2 value we have found
        minChi2 = chi2
        bestFitMz1D = mZ
                
# as we will only be interested in the change in chi2 w.r.t. mZ,  we can rescale all the chi2
#values such that the minimum chi2 value is 0. 
chi2Ar = chi2Ar - minChi2

#we expect the chi2 vs. mZ curve to be quadratic, so let's fit that fucntion to it.
z = np.polyfit(mZAr, chi2Ar, 2) #"2" for a second-order polynomial
p = np.poly1d(z)

#lets plot our chi2 values vs mz along with the fit
plt.figure()
fig, ax = plt.subplots()
ax.plot(mZAr, chi2Ar, 'r+', lw=5, alpha=0.6, label="chi2 evaluations")
ax.plot(mZAr, p(mZAr), 'b-', lw=5, alpha=0.6, label="quadratic fit")

# we can display the estimated uncertianty on mZ via critical values () of the delta chi-squared curve
y0 = 1.0
crit = (p - y0).roots # roots of the polynominal -1, i.e., the mz values where p = 1 

#shading in the uncertainty band 
px=np.arange(crit[1],crit[0],0.001)
ax.fill_between(px,p(px),alpha=0.5, color='g', label="uncertainty")
ax.legend()

#setting reasonable axis ranges and titles
ax.set_xlim(minMz, maxMz)
ax.set_ylim(0.0, 5.0)
plt.xlabel("mZ [MeV]")
plt.ylabel("delta chi-squared")

plt.show()

#print result and uncertainty
my1DResult = bestFitMz1D
my1DUncertainty = bestFitMz1D-crit[1]
print("best-fit = " + str(my1DResult) + " +/- " + str(my1DUncertainty))

# we now compare our data histogram with the cauchy distribution with the mean and standard deviation
# that minimise the chi-squared. This pdf should agree much better than the pdf with the PDG values.

plt.figure()
p = (cauchy.pdf(centres, my1DResult, sigmaZPDG))

pInt = bin_width * sum(p[0:nBins])
scale = integral/pInt
pScaled = (cauchy.pdf(centres, my1DResult, sigmaZPDG) *scale)

plt.plot(centres, pScaled, 'k', linewidth=2)

x = np.linspace(87000, 94000, 1000)
pScaledNorm = (norm.pdf(x, my1DResult, (sigmaZPDG*0.8) )*(0.6*scale))

#plt.plot(x, pScaledNorm, 'r', linewidth=2)

plt.hist(lvArray.mass, bins=nBins, range=[minMass, maxMass], alpha=0.6, color='g')
plt.xlabel("mumu mass [MeV]")
plt.ylabel("# events")
plt.show() 

# Try a double gaussian with curve_fit

In [ ]:
#assume mass, sigma
from scipy.optimize import curve_fit

def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - ((x - mean) / standard_deviation) ** 2)

def doubleGaussian(x, mean1, amplitude1, standard_deviation1, mean2, amplitude2, standard_deviation2):
    gaus1 = amplitude1 * np.exp( - ((x - mean1) / standard_deviation1) ** 2)
    gaus2 = amplitude2 * np.exp( - ((x - mean2) / standard_deviation2) ** 2)
    return gaus1 + gaus2

#bestFitParams, covarianceMatrix = curve_fit(gaussian, centres, countsData, p0=[91000., 2000., 10000.])
bestFitParams, covarianceMatrix = curve_fit(doubleGaussian, centres, countsData, p0=[91000., 2000., 10000., 91000, 6000, 2000])

x_interval_for_fit = np.linspace(edges[0], edges[-1], 10000)
#plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *bestFitParams), label='gaussian fit')
plt.plot(x_interval_for_fit, doubleGaussian(x_interval_for_fit, *bestFitParams),'r-', linewidth=3, label='double gaussian fit')
plt.errorbar(centres, countsData, yerr=np.sqrt(countsData), label="Data", fmt='o', mfc='k', mec='k', mew=0.1, ecolor='k')
plt.legend()
plt.xlabel("$ m_{\mu\mu}$")
plt.ylabel("Data")

now we have a 'measurement' of mZ, let's make a plot comparing our measured value with that from
the PDG to assess how good our measurement is.

In [ ]:
mZPDGAr = np.array([mZPDG]) 
uncZPDGAr = np.array([2.1]) 
yPDGAr = np.array([1.0]) 

mZ1D = np.array([my1DResult]) 
sigZ1D = np.array([my1DUncertainty]) 
y1D = np.array([2.0]) 

plt.figure()
ax = plt.gca()
ax.set_xlim(minMz, maxMz+700)#need to extend x axis range to include PDG value
ax.set_ylim(0.0, 4.0)

plt.errorbar(mZPDGAr, yPDGAr, xerr=uncZPDGAr, label="Particle Data Group", elinewidth=5,  fmt="o", mfc="blue", ms=8)
plt.errorbar(mZ1D, y1D, xerr=sigZ1D, fmt="o", label="ATLAS Open Data", elinewidth=5, ms=8, mfc='blue')

plt.xlabel("mZ [MeV]")
plt.yticks(y1D, " ")


ax.axes.yaxis.set_visible(False)

plt.legend()
plt.show()

Thats completes steps 1-5. You have seen how the chi-squared minisation technique allows us to extract measurements
of a physical parameter such as $m_{Z}$ by comparing the predictions of statistical model to data.

# Now it's your turn!

Refer back to the lab manual to see how you might improve this measurement and have fun!